In [3]:
!pip install chromadb
!pip install google.generativeai
!pip install langchain-google-genai
!pip install langchain
!pip install langchain_community


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 9.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7

In [4]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 4.6 MB/s eta 0:00:00


In [9]:
import os
from dotenv import load_dotenv

env_path = '/content/drive/My Drive/llmEnv/.env'
load_dotenv(env_path)

# make a .env path containing following two values
# GOOGLE_API_KEY="Get your own key"
# LAST_PROCESSED_TIME='1970-01-01 00:00:00'

True

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
# Template and model
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

template = """Given the following context, please provide a detailed and accurate answer to the question.
Context: {context}
Question: {input}
Answer:"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])
llm = ChatGoogleGenerativeAI(model="gemini-pro")

In [37]:
import os, datetime
from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import PyPDFLoader
from dotenv import set_key
from datetime import datetime

last_processed_time = os.getenv('LAST_PROCESSED_TIME', '1970-01-01 00:00:00')
last_processed_time = datetime.strptime(last_processed_time, '%Y-%m-%d %H:%M:%S')
print("last _processed _time {}".format(last_processed_time))

docs_folder_pdf = '/content/drive/My Drive/llmctxpdf'
documents_pdf = []

docs_folder_text = '/content/drive/My Drive/llmctx'
documents_text = []

def get_modification_time(filepath):
    return datetime.fromtimestamp(os.path.getmtime(filepath))

local_mod_time = last_processed_time
print("last mod _time {}".format(last_processed_time))

for filename in os.listdir(docs_folder_text):
    if filename.endswith('.txt'):
        # print(filename)
        filepath = os.path.join(docs_folder_text, filename)
        file_mod_time = get_modification_time(filepath)
        if file_mod_time > last_processed_time:
          loader = TextLoader(filepath)
          documents_text.extend(loader.load_and_split())
        if file_mod_time > local_mod_time:
          local_mod_time = file_mod_time

for filename in os.listdir(docs_folder_pdf):
    if filename.endswith('.pdf'):
        # print(filename)
        filepath = os.path.join(docs_folder_pdf, filename)
        file_mod_time = get_modification_time(filepath)
        if file_mod_time > last_processed_time:
          loader = PyPDFLoader(filepath)
          documents_pdf.extend(loader.load_and_split())
        if file_mod_time > local_mod_time:
          local_mod_time = file_mod_time

os.environ['LAST_PROCESSED_TIME'] = local_mod_time.strftime('%Y-%m-%d %H:%M:%S')
set_key(env_path, 'LAST_PROCESSED_TIME', local_mod_time.strftime('%Y-%m-%d %H:%M:%S'))

last _processed _time 2024-06-09 14:34:10
last mod _time 2024-06-09 14:34:10


(True, 'LAST_PROCESSED_TIME', '2024-06-10 08:35:08')

In [39]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma

# embedding_txt = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
# embedding_pdf = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# db_text = Chroma.from_documents(documents_text, embedding_txt)
# db_pdf = Chroma.from_documents(documents_pdf, embedding_pdf)

# Initialize embeddings
embedding_txt = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
embedding_pdf = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Paths to save Chroma databases
db_path_text = '/content/drive/My Drive/llmEnv/chroma_text_db/'
db_path_pdf = '/content/drive/My Drive/llmEnv/chroma_pdf_db/'

# Load or create Chroma database for text documents
if os.path.exists(db_path_text):
    db_text = Chroma(persist_directory=db_path_text, embedding_function=embedding_txt)
else:
    db_text = Chroma.from_documents(documents=documents_text, embedding=embedding_txt, persist_directory=db_path_text)
    documents_text = []

# Load or create Chroma database for PDF documents
if os.path.exists(db_path_pdf):
    db_pdf = Chroma(persist_directory=db_path_pdf, embedding_function=embedding_pdf)
else:
    db_pdf = Chroma.from_documents(documents=documents_pdf, embedding=embedding_pdf, persist_directory=db_path_pdf)
    documents_pdf = []

# Add new documents to the databases
if documents_text:
    db_text.add_documents(documents_text)
if documents_pdf:
    db_pdf.add_documents(documents_pdf)

# Save the updated databases
db_text.persist()
db_pdf.persist()

In [40]:
retriever_text = db_text.as_retriever(search_kwargs={"k": 5})
retriever_pdf = db_pdf.as_retriever(search_kwargs={"k": 5})


In [41]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

combine_docs_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever_text, combine_docs_chain)

In [50]:
response = retrieval_chain.invoke({"input" : "What is full form of IPL"})
print(response["answer"])
#print(response)

This context does not mention anything about the full form of IPL, so I cannot answer this question from the provided context.
